<a href="https://colab.research.google.com/github/sagarsood01/Chat-Analyzer/blob/main/chat%20analyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()
!pip install pandas matplotlib wordcloud emoji

import re
import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import emoji
from collections import Counter

file_path = "_chat 2.txt"

with open(file_path, 'r', encoding='utf-8') as f:
    data = f.read()

pattern = r'\[(\d{1,2}/\d{1,2}/\d{2,4}), (\d{1,2}:\d{2}:\d{2}\s?[APap][Mm])\]\s([^:]+):\s(.*)'
messages = re.findall(pattern, data)

df = pd.DataFrame(messages, columns=['Date', 'Time', 'User', 'Message'])
df['Date'] = pd.to_datetime(df['Date'], errors='coerce', dayfirst=True)
df.dropna(subset=['Date'], inplace=True)

print("Chat successfully loaded!")
print(df.head())

print("\nChat Summary:")
print(f"Total messages: {len(df)}")
print(f"Total users: {df['User'].nunique()}")
print("Users:", list(df['User'].unique()))

active_users = df['User'].value_counts()
print("\nMost Active Users:")
print(active_users)

plt.figure(figsize=(8,4))
active_users.plot(kind='bar')
plt.title('Most Active Users')
plt.xlabel('User')
plt.ylabel('Message Count')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

daily_msgs = df.groupby('Date').size()
plt.figure(figsize=(8,4))
daily_msgs.plot(kind='line')
plt.title('Daily Message Trend')
plt.xlabel('Date')
plt.ylabel('Messages per Day')
plt.tight_layout()
plt.show()

def clean_text(msg):
    stopwords = ['to','the','and','a','i','you','of','is','it','in','for','on','that','this','me']
    words = [w.lower() for w in re.findall(r'\b\w+\b', msg) if w.lower() not in stopwords]
    return words

words = df['Message'].apply(clean_text)
flat_words = [w for sub in words for w in sub]
word_freq = Counter(flat_words).most_common(15)

print("\nMost Common Words:")
for w, f in word_freq:
    print(f"{w}: {f}")

plt.figure(figsize=(8,4))
plt.bar([w for w,_ in word_freq], [f for _,f in word_freq])
plt.title('Most Common Words')
plt.xticks(rotation=45)
plt.show()

all_text = ' '.join(df['Message'])
wc = WordCloud(width=800, height=400, background_color='white').generate(all_text)

plt.figure(figsize=(10,5))
plt.imshow(wc, interpolation='bilinear')
plt.axis('off')
plt.title('Word Cloud of Chat Messages')
plt.show()

def extract_emojis(s):
    return [c for c in s if c in emoji.EMOJI_DATA]

emojis_list = sum(df['Message'].apply(extract_emojis), [])
emoji_count = Counter(emojis_list).most_common(10)

print("\nMost Used Emojis:")
for em, cnt in emoji_count:
    print(f"{em}: {cnt}")

if emoji_count:
    plt.figure(figsize=(6,4))
    plt.bar([em for em,_ in emoji_count], [cnt for _,cnt in emoji_count])
    plt.title('Most Used Emojis')
    plt.show()
else:
    print("No emojis found in chat.")
